# Quality of life, MCI metrics

## Measure change of life behaviours

* Activities of Daily Living
* Mobility
* Sociability
* *Cognition* (not measured)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from reasoning.presets.dataset import load_default_datasets

In [3]:
import pandas as pd
import numpy as np

from bokeh.plotting import output_notebook

from reasoning.data_analysis.data_viz import plot
from reasoning.data_analysis.update_csv import full_csv_update
from reasoning.data_analysis.data_toolbox import load_activities, load_events, load_groundtruth
from reasoning.presets.dataset import load_default_datasets

from datetime import timedelta

output_notebook()

Loading BokehJS ...

In [8]:
# events, activities, groundtruth = load_default_datasets(update_csv_files=True)

path_to_dataset_E = '../new_dataset/Dataset-Extend'
path_to_dataset_N = '../new_dataset/Dataset-Normandie'

events_N, activities_N = load_default_datasets(update_csv_files=False,
                    path=path_to_dataset_N,
                    event_dataset='public.event.csv',
                    activity_dataset='public.activity.csv',
                    delim=',',
                    patients='N')
events_E, activities_E = load_default_datasets(update_csv_files=False,
                    path=path_to_dataset_E,
                    event_dataset='public.event.csv',
                    activity_dataset='public.activity.csv',
                    delim=';',
                    patients='E')

Loading events...
Loading activities...
Cleaning up events...
Cleaning up activities...
To choose patients from N
{'A': (11, None, None), 'C': (13, None, None), 'B': (12, None, None), 'E': (15, None, None), 'D': (14, None, None)}
Updated patients
Updated activities
Renamed activities
Loading events...
Loading activities...
Cleaning up events...
Cleaning up activities...
To choose patients from E
{'A': (34, None, None), 'C': (21, None, None), 'B': (20, None, None), 'E': (23, None, None), 'D': (22, None, None)}
Updated patients
Updated activities
Renamed activities


In [15]:
events_N.tail()

id               createdAt  \
patient date                                                  
E       2017-01-11 13:07:06  793703  2017-01-11 14:07:07+01   
        2017-01-11 13:07:13  793704  2017-01-11 14:07:14+01   
        2017-01-11 13:08:15  793705  2017-01-11 14:08:16+01   
        2017-01-11 13:27:15  793706  2017-01-11 14:27:17+01   
        2017-01-11 13:28:21  793707  2017-01-11 14:28:22+01   

                                          updatedAt           sensor value  
patient date                                                                
E       2017-01-11 13:07:06  2017-01-11 14:07:07+01  Bathroom Motion    On  
        2017-01-11 13:07:13  2017-01-11 14:07:14+01  Bathroom Motion    On  
        2017-01-11 13:08:15  2017-01-11 14:08:16+01  Bathroom Motion   Off  
        2017-01-11 13:27:15  2017-01-11 14:27:17+01   Bedroom Motion    On  
        2017-01-11 13:28:21  2017-01-11 14:28:22+01   Bedroom Motion   Off

In [19]:
print(events_E.shape, events_N.shape)

merged = pd.concat([events_E, events_N])

print(merged.shape)
merged.head()


((86242, 5), (427651, 5))
(513893, 5)


id               createdAt  \
patient date                                                  
B       2017-01-11 14:26:40  315611  2017-01-11 14:26:41+00   
        2017-01-11 14:26:54  315615  2017-01-11 14:26:55+00   
        2017-01-11 14:26:59  315616  2017-01-11 14:26:59+00   
        2017-01-11 14:27:01  315618  2017-01-11 14:27:01+00   
        2017-01-11 14:27:38  315622  2017-01-11 14:27:38+00   

                                          updatedAt           sensor value  
patient date                                                                
B       2017-01-11 14:26:40  2017-01-11 14:26:41+00  Bathroom Motion   Off  
        2017-01-11 14:26:54  2017-01-11 14:26:55+00  Bathroom Motion    On  
        2017-01-11 14:26:59  2017-01-11 14:26:59+00  Bathroom Motion    On  
        2017-01-11 14:27:01  2017-01-11 14:27:01+00  Bathroom Motion    On  
        2017-01-11 14:27:38  2017-01-11 14:27:38+00  Bathroom Motion    On

## Energy rate

In [ ]:
events.reset_index(level=0, drop=True)

In [ ]:
events.loc['C']

In [ ]:
# TODO: Make it work as groupby

def aggregate_events(events):
    aggregated_events = events.reset_index(level=0, drop=True).value.resample('D', how='count')
    aggregated_events = aggregated_events[aggregated_events != 0]
    min_events, max_events = min(aggregated_events), max(aggregated_events)
    aggregated_events = aggregated_events * 100 / max_events
    return aggregated_events

events.groupby(level=0).apply(aggregate_events)

In [ ]:
aggregated_events = events.loc['C'].value.resample('D', how='count')
min_events, max_events = min(aggregated_events), max(aggregated_events)
aggregated_events = aggregated_events * 100 / max_events
aggregated_events

In [ ]:
def custom(my_day):
    a = my_day.copy()
    a.loc[:, 'duration'] = a.index - a.since
    falls = a[(a.activity == "Fall") & (a.duration >= timedelta(minutes=5))]
    return not falls.empty

a = activities.loc['C']
fall_per_day = a.resample('D', how=custom).activity
fall_days = fall_per_day[fall_per_day].index

In [ ]:
from bokeh.plotting import figure, show

p = figure(title="Amount of activity per day", x_axis_label='Day', y_axis_label='Events', x_axis_type="datetime",
          tools="box_select,xpan,xwheel_zoom,reset,save", height=400)

p.line(aggregated_events.index, aggregated_events.values)
p.segment(x0=fall_days, y0=5,
          x1=fall_days + timedelta(days=1), y1=5, line_width=10, legend="Falls", color='red')

show(p)

## Time outdoor / Time indoor

In [ ]:
activities.loc['I'].activity.unique()

In [ ]:
def outside_time_estimator(my_day):
    outside = my_day[my_day.activity == "Go Out"]
    time_spend_outside = outside.reset_index().apply(lambda x: x.date - x.since, axis=1)
    if time_spend_outside.empty:
        return timedelta(seconds=0)
    return time_spend_outside.sum()

a = activities.loc['J']
time_spend_outside = a.resample("D", how=outside_time_estimator).activity
time_spend_outside

max_bound = time_spend_outside[time_spend_outside != 0].quantile(.9)
time_spend_outside.loc[time_spend_outside > max_bound] = max_bound

In [ ]:
from bokeh.plotting import figure, show

p = figure(title="Time spent outside per day", x_axis_label='Day', y_axis_label='Outside time',
           x_axis_type="datetime",
           tools="box_select,xpan,xwheel_zoom,reset,save", height=400)

p.yaxis[0].formatter = NumeralTickFormatter(format="00:00:00")

p.line(time_spend_outside.index, (time_spend_outside / np.timedelta64(1, 's')).values)
# p.segment(x0=fall_days, y0=5,
#           x1=fall_days + timedelta(days=1), y1=5, line_width=10, legend="Falls", color='red')

show(p)

# Time spent in bathroom

In [ ]:
from dateutil.parser import parse

def toilet_duration_estimator(my_day):
    outside = my_day[my_day.activity == "Go Toilet"]
    time_spend_outside = outside.reset_index().apply(lambda x: x.date - x.since, axis=1)
    if time_spend_outside.empty:
        return timedelta(seconds=0)
    return time_spend_outside.sum()

def toilet_count_estimator(my_day):
    outside = my_day[my_day.activity == "Go Toilet"]
    return outside.count()

a = activities.loc['J']
duration_spend_in_toilet = a.resample("D", how=toilet_duration_estimator).activity
duration_spend_in_toilet

max_bound = duration_spend_in_toilet[duration_spend_in_toilet != 0].quantile(.9)
duration_spend_in_toilet.loc[duration_spend_in_toilet > max_bound] = max_bound

toilet_occurences = a.resample("D", how=toilet_count_estimator).activity
toilet_occurences

max_bound = toilet_occurences[toilet_occurences != 0].quantile(.9)
toilet_occurences.loc[toilet_occurences > max_bound] = max_bound

In [ ]:
from bokeh.plotting import figure, show
from bokeh.models import LinearAxis, Range1d

from bokeh.models.formatters import NumeralTickFormatter

p = figure(title="Time spent in toilet per day", x_axis_label='Day', y_axis_label='Toilet time',
           x_axis_type="datetime",
           tools="box_select,xpan,xwheel_zoom,reset,save", height=400)

p.yaxis[0].formatter = NumeralTickFormatter(format="00:00:00")

p.extra_y_ranges = {"occurences": Range1d(start=0, end=toilet_occurences.max())}
p.add_layout(LinearAxis(y_range_name="occurences"), 'right')

p.line(duration_spend_in_toilet.index, (duration_spend_in_toilet / np.timedelta64(1, 's')).values)

p.line(toilet_occurences.index, toilet_occurences.values, color="Red", y_range_name="occurences")
# p.segment(x0=fall_days, y0=5,
#           x1=fall_days + timedelta(days=1), y1=5, line_width=10, legend="Falls", color='red')

show(p)

## Toilet by night

In [ ]:
hours = a.index.get_level_values('date').map(lambda x: x.hour)
hours
a2 = a[(hours >= 20) | (hours < 10)].copy()
a2.loc[:, 'shifted_hours'] = a2.index.get_level_values('date') + timedelta(hours=4)
a2.loc[:, 'since'] = a2.since + timedelta(hours=4)
a2.reset_index(drop=True, inplace=True)
a2.rename(columns={'shifted_hours': 'date'}, inplace=True)
a2.set_index('date', inplace=True)

max_bound = duration_spend_in_toilet[duration_spend_in_toilet != 0].quantile(.9)
duration_spend_in_toilet.loc[duration_spend_in_toilet > max_bound] = max_bound

duration_spend_in_toilet_by_night = a2.resample("D", how=toilet_duration_estimator).activity
duration_spend_in_toilet_by_night

max_bound = duration_spend_in_toilet_by_night[duration_spend_in_toilet_by_night != 0].quantile(.9)
duration_spend_in_toilet_by_night.loc[duration_spend_in_toilet_by_night > max_bound] = max_bound

toilet_occurences_by_night = a2.resample("D", how=toilet_count_estimator).activity
toilet_occurences_by_night

max_bound = toilet_occurences_by_night[toilet_occurences_by_night != 0].quantile(.9)
toilet_occurences_by_night.loc[toilet_occurences_by_night > max_bound] = max_bound

In [ ]:
from bokeh.plotting import figure, show
from bokeh.models import LinearAxis, Range1d

p = figure(title="Time spent in toilet per night", x_axis_label='Day', y_axis_label='Toilet time',
           x_axis_type="datetime",
           tools="box_select,xpan,xwheel_zoom,reset,save", height=400)

p.yaxis[0].formatter = NumeralTickFormatter(format="00:00:00")

p.extra_y_ranges = {"occurences": Range1d(start=0, end=toilet_occurences_by_night.max())}
p.add_layout(LinearAxis(y_range_name="occurences"), 'right')

p.line(duration_spend_in_toilet_by_night.index, (duration_spend_in_toilet_by_night / np.timedelta64(1, 's')).values)

p.line(toilet_occurences_by_night.index, toilet_occurences_by_night.values, color="Red", y_range_name="occurences")
# p.segment(x0=fall_days, y0=5,
#           x1=fall_days + timedelta(days=1), y1=5, line_width=10, legend="Falls", color='red')

show(p)

# Kitchen

In [ ]:
def kitchen_duration_estimator(my_day):
    in_kitchen = my_day[my_day.activity.isin(['Eat Meal', 'Cook Meal'])]
    time_spend_in_kitchen = in_kitchen.reset_index().apply(lambda x: x.date - x.since, axis=1)
    if time_spend_in_kitchen.empty:
        return timedelta(seconds=0)
    return time_spend_in_kitchen.sum()

def kitchen_count_estimator(my_day):
    in_kitchen = my_day[my_day.activity.isin(['Eat Meal', 'Cook Meal'])]
    return in_kitchen.count()

a = activities.loc['J']
duration_spend_in_kitchen = a.resample("D", how=kitchen_duration_estimator).activity
duration_spend_in_kitchen

max_bound = duration_spend_in_kitchen[duration_spend_in_kitchen != 0].quantile(.9)
duration_spend_in_kitchen.loc[duration_spend_in_kitchen > max_bound] = max_bound

kitchen_occurences = a.resample("D", how=kitchen_count_estimator).activity
kitchen_occurences

max_bound = kitchen_occurences[kitchen_occurences != 0].quantile(.9)
kitchen_occurences.loc[kitchen_occurences > max_bound] = max_bound

In [ ]:
from bokeh.plotting import figure, show
from bokeh.models import LinearAxis, Range1d

p = figure(title="Time spent in kitchen per day", x_axis_label='Day', y_axis_label='Kitchen time',
           x_axis_type="datetime",
           tools="box_select,xpan,xwheel_zoom,reset,save", height=400)

p.yaxis[0].formatter = NumeralTickFormatter(format="00:00:00")

p.extra_y_ranges = {"occurences": Range1d(start=0, end=kitchen_occurences.max())}
p.add_layout(LinearAxis(y_range_name="occurences"), 'right')

p.line(duration_spend_in_kitchen.index, (duration_spend_in_kitchen / np.timedelta64(1, 's')).values)

p.line(kitchen_occurences.index, kitchen_occurences.values, color="Red", y_range_name="occurences")
# p.segment(x0=fall_days, y0=5,
#           x1=fall_days + timedelta(days=1), y1=5, line_width=10, legend="Falls", color='red')

show(p)

# Sleeping

In [ ]:
def sleeping_duration_estimator(my_day):
    sleeping = my_day[my_day.activity == 'Sleep']
    time_spend_sleeping = sleeping.reset_index().apply(lambda x: x.date - x.since, axis=1)
    if time_spend_sleeping.empty:
        return timedelta(seconds=0)
    return time_spend_sleeping.sum()

def sleeping_count_estimator(my_day):
    sleeping = my_day[my_day.activity == 'Sleep']
    return sleeping.count()

def nap_duration_estimator(my_day):
    sleeping = my_day[my_day.activity == 'Nap']
    time_spend_sleeping = sleeping.reset_index().apply(lambda x: x.date - x.since, axis=1)
    if time_spend_sleeping.empty:
        return timedelta(seconds=0)
    return time_spend_sleeping.sum()

def nap_count_estimator(my_day):
    sleeping = my_day[my_day.activity == 'Nap']
    return sleeping.count()

a = activities.loc['J']
duration_spend_sleeping = a.resample("D", how=sleeping_duration_estimator).activity
duration_spend_sleeping

sleeping_occurences = a.resample("D", how=sleeping_count_estimator).activity
sleeping_occurences

a = activities.loc['J']
duration_spend_nap = a.resample("D", how=nap_duration_estimator).activity
duration_spend_nap

nap_occurences = a.resample("D", how=nap_count_estimator).activity
nap_occurences

In [ ]:
sleep_df = pd.DataFrame(data={'nap': duration_spend_nap.values, 'sleep': duration_spend_sleeping.values},
                        index=duration_spend_nap.index)

In [ ]:
def stacked(df, categories):
    areas = dict()
    last = np.zeros(len(df[categories[0]]))
    for cat in categories:
#         print(last, df[cat])
        next = last + df[cat] / np.timedelta64(1, 's')
        areas[cat] = np.hstack((last[::-1], next))
        last = next
    return areas

In [ ]:
brewer['Spectral'][8]

In [ ]:
from bokeh.palettes import brewer

p = figure(title="Time spent sleeping per day", x_axis_label='Day', y_axis_label='Sleeping time',
           x_axis_type="datetime",
           tools="box_select,xpan,xwheel_zoom,reset,save", height=400)

# p.extra_y_ranges = {"occurences": Range1d(start=0, end=sleeping_occurences.max())}
# p.add_layout(LinearAxis(y_range_name="occurences"), 'right')

p.yaxis[0].formatter = NumeralTickFormatter(format="00:00:00")

x2 = np.hstack((sleep_df.index[::-1], sleep_df.index))

categories = ['nap', 'sleep']
areas = stacked(sleep_df, categories)

colors = ['#fee08b', '#fdae61']

p.patches([x2] * len(areas), [areas[cat] for cat in categories],
          color=colors, alpha=0.8, line_color=None)

# p.line(duration_spend_sleeping.index, (duration_spend_sleeping / np.timedelta64(1, 's')).values)

# p.line(sleeping_occurences.index, sleeping_occurences.values, color="Red", y_range_name="occurences")

show(p)

# Active

In [ ]:
def active_duration_estimator(my_day):
    active = my_day[~my_day.activity.isin(['Go Out', 'Sleep', 'Nap', 'Inactive'])]
    time_spend_active = active.reset_index().apply(lambda x: x.date - x.since, axis=1)
    if time_spend_active.empty:
        return timedelta(seconds=0)
    return time_spend_active.sum()

a = activities.loc['J']
duration_active = a.resample("D", how=active_duration_estimator).activity
duration_active

max_bound = duration_active[duration_active != 0].quantile(.9)
duration_active.loc[duration_active > max_bound] = max_bound

In [ ]:
from bokeh.plotting import figure, show

p = figure(title="Time spent being active per day", x_axis_label='Day', y_axis_label='Active time',
           x_axis_type="datetime",
           tools="box_select,xpan,xwheel_zoom,reset,save", height=400)

p.yaxis[0].formatter = NumeralTickFormatter(format="00:00:00")

p.line(duration_active.index, (duration_active / np.timedelta64(1, 's')).values)
# p.segment(x0=fall_days, y0=5,
#           x1=fall_days + timedelta(days=1), y1=5, line_width=10, legend="Falls", color='red')

show(p)

## Changing rooms

In [ ]:
def change_rooms_estimator(my_day):
    e = my_day[my_day.value == "On"].copy()
    e.loc[:, 'previous_location'] = e.sensor.shift()
    e = e[(e.sensor != e.previous_location)]
    return e.count()

e = events.loc['J']
change_rooms = e.resample("D", how=change_rooms_estimator).sensor
change_rooms

In [ ]:
from bokeh.plotting import figure, show

p = figure(title="Numer of times changing rooms per day", x_axis_label='Day', y_axis_label='Changing rooms',
           x_axis_type="datetime",
           tools="box_select,xpan,xwheel_zoom,reset,save", height=400)

p.line(change_rooms.index, change_rooms.values)
# p.segment(x0=fall_days, y0=5,
#           x1=fall_days + timedelta(days=1), y1=5, line_width=10, legend="Falls", color='red')

show(p)

# Misc.

In [ ]:
plot(activities=activities, patients=['J'])

In [ ]:
events.loc['J']

In [ ]:
plot(events=events.head(1000))